# Photos of yeast growth for Y2H

- [X] Get the well info
- [X] Make yeast pictures
- [X] Make a matrix showing growth score
- [X] Make a matrix showing final calls

In [8]:
import os

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
import tqdm

import ccsblib
from ccsblib import ccsbplotlib as cplt
from crop_y2h_images import yeast_spot_grid

from plotting import y2h_ppi_per_tf_gene_plot, isoform_display_name
from data_loading import load_isoform_and_paralog_y2h_data

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 50)

In [19]:
y2h = load_isoform_and_paralog_y2h_data()
y2h = y2h.sort_values(['ad_gene_symbol', 'ad_clone_acc', 'db_gene_symbol'])
qry = """SELECT *
           FROM tf_screen.tf_isoform_final
          WHERE category = 'ppi';"""
df = pd.read_sql(qry, ccsblib.paros_connection()).drop(columns=['category'])
y2h = pd.merge(y2h, 
               df, 
               how='left', 
               on=['ad_orf_id', 'db_orf_id'])
y2h.head()

,category,ad_orf_id,ad_clone_acc,ad_gene_symbol,db_orf_id,db_gene_symbol,score,standard_batch,retest_pla,retest_pos,is_partner_category_DNA_polymerase,is_partner_category_DNA_repair,is_partner_category_DNA_replication,is_partner_category_RNA_editing,is_partner_category_RNA_processing,is_partner_category_RNA_turnover,is_partner_category_TF,is_partner_category_cell_cycle,is_partner_category_cell-cell_signaling,is_partner_category_cofactor,is_partner_category_cytoskeleton,is_partner_category_membrane_protein,is_partner_category_metabolism,is_partner_category_nuclear_transport,is_partner_category_other,is_partner_category_protein_folding,is_partner_category_protein_traffiking,is_partner_category_protein_turnover,is_partner_category_signaling,is_partner_category_translation,is_partner_category_vesicle_transport,ad_symbol,3at_repeat_1_score,3at_repeat_2_score,3at_repeat_3_score,chx_repeat_1_score,chx_repeat_2_score,chx_repeat_3_score,lw_repeat_1_score,lw_repeat_2_score,lw_repeat_3_score,avg_at,avg_cx,avg_lw,growth_score,seq1,seq2,final_seq,final_score,retest_id
0,non_paralog_control,100093,ARNT2|1/6|08C12,ARNT2,53886,AES,1,TFr08,94,F07,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tf_paralog_ppis,100093,ARNT2|1/6|08C12,ARNT2,11222,ARNTL,0,TFr08,61,G10,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tf_paralog_ppis,100093,ARNT2|1/6|08C12,ARNT2,56620,ARNTL2,0,TFr08,105,E08,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,non_paralog_control,100093,ARNT2|1/6|08C12,ARNT2,6534,BANP,NS,TFr08,6,B03,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tf_paralog_ppis,100093,ARNT2|1/6|08C12,ARNT2,54316,CLOCK,1,TFr08,43,H04,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
from matplotlib import patches

def growth_score_grid(df, ax=None, fontsize=12):
    if ax is None:
        ax = plt.gca()
    rows = df['ad_clone_acc'].unique()
    columns = df['db_gene_symbol'].unique()
    for i_row, row in enumerate(rows):
        for i_column, column in enumerate(columns):
            if df.loc[(df['ad_clone_acc'] == row) & (df['db_gene_symbol'] == column), 'score'].values[0] == 'NS':
                rect = patches.Rectangle((i_column / len(columns), i_row / len(rows)),
                                         1 / len(columns),
                                         1 / len(rows),
                                         linewidth=3,
                                         edgecolor='r',
                                         facecolor='none')
                ax.add_patch(rect)
            for i_media, media in enumerate(['lw', 'chx', '3at']):
                for i_rep, rep in enumerate(['1', '2', '3']):
                    gs = df.loc[(df['ad_clone_acc'] == row) &
                                (df['db_gene_symbol'] == column),
                                media + '_repeat_' + rep + '_score'].values[0]
                    ax.text(i_column / len(columns) + (i_rep + 1) / (5 * len(columns)),
                            i_row / len(rows) + (i_media + 1) / (5 * len(rows)),
                            str('{:.2g}'.format(gs)),
                            ha='center',
                            va='center',
                            fontsize=fontsize)
    ax.invert_yaxis()
    ax.axis('off')


def per_tf_y2h_yeast_images(tf_gene):
    selections={    'ad_clone_acc': y2h.loc[(y2h['category'] == 'tf_isoform_ppis') &
                                       (y2h['ad_gene_symbol'] == tf_gene),
                                       'ad_clone_acc'].unique(),
                    'db_gene_symbol': y2h.loc[(y2h['category'] == 'tf_isoform_ppis') &
                                       (y2h['ad_gene_symbol'] == tf_gene),
                                       'db_gene_symbol'].unique(),
                    'media': ['LW', 'CX', 'AT'],
                    'rep': ['A', 'B', 'C']} 
    labels={'media': ['-LW', 'CHX', '3AT'],  #['SC -LW', 'SC -LH +3AT +CHX', 'SC -LWH +3AT'],
            'rep': [],
            'ad_clone_acc': [isoform_display_name(clone_acc) for clone_acc in selections['ad_clone_acc']]}
    n_column = len(selections['db_gene_symbol'])
    n_row = len(selections['ad_clone_acc'])
    if n_row < n_column:
        fig, axes = plt.subplots(3, 1)
        fig.set_size_inches((n_column + 1.2) * 0.5, (n_row * 3 + 1.5) * 0.5)
    else:
        fig, axes = plt.subplots(1, 3)
        fig.set_size_inches((n_column * 3 + 1.2) * 0.5, (n_row + 1.5) * 0.5)
    yeast_spot_grid(data=y2h.loc[(y2h['category'] == 'tf_isoform_ppis') &
                                 (y2h['ad_gene_symbol'] == tf_gene), :],
                     variables=selections,
                     file_structure='{standard_batch}.{media}.rep{rep}',
                     well_img_dir='/Users/lukelambourne/Work/y2h_cropped_images/individual_cropped_images',
                     labels=labels,
                     ax=axes[0])
    growth_score_grid(y2h.loc[(y2h['category'] == 'tf_isoform_ppis') &
                     (y2h['ad_gene_symbol'] == tf_gene), :],
                      ax=axes[1],
                      fontsize=6)
    axes[1].set_aspect(max(axes[0].get_ylim()) / max(axes[0].get_xlim()))    
    y2h_ppi_per_tf_gene_plot(tf_gene,
                             data=y2h,
                             ax=axes[2])
    if n_row < n_column:
        axes[2].set_xticks([])
        plt.subplots_adjust(hspace=0.2)
    else:
        axes[2].set_yticks([])
        plt.subplots_adjust(wspace=0.2)
    
    fig_dir = '../../figures/y2h_yeast_images'
    os.makedirs(fig_dir, exist_ok=True)
    plt.savefig(os.path.join(fig_dir, tf_gene + '_y2h_yeast_images_and_results.pdf'),
                bbox_inches='tight',
                dpi=600)
    plt.close(fig)

#per_tf_y2h_yeast_images('CREBZF')

In [87]:
for tf_gene in tqdm.tqdm(y2h.loc[(y2h['category'] == 'tf_isoform_ppis'), 'ad_gene_symbol'].unique()):
    per_tf_y2h_yeast_images(tf_gene)



  0%|          | 0/164 [00:00<?, ?it/s]

  1%|          | 1/164 [00:11<30:58, 11.40s/it]

  1%|          | 2/164 [00:12<22:29,  8.33s/it]

  2%|▏         | 3/164 [00:14<17:15,  6.43s/it]

  2%|▏         | 4/164 [00:18<15:32,  5.83s/it]

  3%|▎         | 5/164 [00:19<11:21,  4.29s/it]

  4%|▎         | 6/164 [00:24<11:22,  4.32s/it]

  4%|▍         | 7/164 [00:29<11:53,  4.54s/it]

  5%|▍         | 8/164 [00:30<09:25,  3.63s/it]

  5%|▌         | 9/164 [00:31<07:16,  2.82s/it]

  6%|▌         | 10/164 [00:34<07:04,  2.76s/it]

  7%|▋         | 11/164 [00:41<10:13,  4.01s/it]

  7%|▋         | 12/164 [00:42<08:19,  3.29s/it]

  8%|▊         | 13/164 [00:44<07:03,  2.80s/it]

  9%|▊         | 14/164 [00:49<08:26,  3.38s/it]

  9%|▉         | 15/164 [00:50<06:54,  2.78s/it]

 10%|▉         | 16/164 [00:51<05:20,  2.16s/it]

 10%|█         | 17/164 [00:52<04:43,  1.93s/it]

 11%|█         | 18/164 [00:59<08:23,  3.45s/it]

 12%|█▏        | 19/164 [01:04<09:18,  3.85s/it]

 12%|█▏        |

 88%|████████▊ | 145/164 [16:53<00:37,  1.98s/it]

 89%|████████▉ | 146/164 [16:55<00:33,  1.87s/it]

 90%|████████▉ | 147/164 [16:59<00:43,  2.54s/it]

 90%|█████████ | 148/164 [17:02<00:41,  2.61s/it]

 91%|█████████ | 149/164 [17:03<00:31,  2.09s/it]

 91%|█████████▏| 150/164 [17:04<00:24,  1.76s/it]

 92%|█████████▏| 151/164 [17:14<00:53,  4.15s/it]

 93%|█████████▎| 152/164 [17:14<00:37,  3.09s/it]

 93%|█████████▎| 153/164 [19:15<07:02, 38.41s/it]

 94%|█████████▍| 154/164 [19:19<04:39, 27.95s/it]

 95%|█████████▍| 155/164 [19:19<02:58, 19.82s/it]

 95%|█████████▌| 156/164 [19:21<01:54, 14.29s/it]

 96%|█████████▌| 157/164 [19:22<01:11, 10.25s/it]

 96%|█████████▋| 158/164 [19:22<00:43,  7.31s/it]

 97%|█████████▋| 159/164 [19:23<00:26,  5.31s/it]

 98%|█████████▊| 160/164 [19:34<00:28,  7.03s/it]

 98%|█████████▊| 161/164 [19:34<00:15,  5.11s/it]

 99%|█████████▉| 162/164 [19:36<00:08,  4.07s/it]

 99%|█████████▉| 163/164 [19:40<00:03,  3.98s/it]

100%|██████████| 164/164 [19:45

In [29]:
y2h.loc[y2h['ad_clone_acc'].str.startswith('ARNT2|2') &
        (y2h['db_gene_symbol'] == 'PID1'), :]

,category,ad_orf_id,ad_clone_acc,ad_gene_symbol,db_orf_id,db_gene_symbol,score,standard_batch,retest_pla,retest_pos,is_partner_category_DNA_polymerase,is_partner_category_DNA_repair,is_partner_category_DNA_replication,is_partner_category_RNA_editing,is_partner_category_RNA_processing,is_partner_category_RNA_turnover,is_partner_category_TF,is_partner_category_cell_cycle,is_partner_category_cell-cell_signaling,is_partner_category_cofactor,is_partner_category_cytoskeleton,is_partner_category_membrane_protein,is_partner_category_metabolism,is_partner_category_nuclear_transport,is_partner_category_other,is_partner_category_protein_folding,is_partner_category_protein_traffiking,is_partner_category_protein_turnover,is_partner_category_signaling,is_partner_category_translation,is_partner_category_vesicle_transport
1131,tf_isoform_ppis,101247,ARNT2|2/6|09E01,ARNT2,11285,PID1,1,TFr07,13,A03,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [26]:
# CREB5-2 KRTAP10-8
y2h.loc[y2h['ad_clone_acc'].str.startswith('CREB5|2') &
        (y2h['db_gene_symbol'] == 'KRTAP10-8'), :]

,category,ad_orf_id,ad_clone_acc,ad_gene_symbol,db_orf_id,db_gene_symbol,score,standard_batch,retest_pla,retest_pos,...,is_partner_category_membrane_protein,is_partner_category_metabolism,is_partner_category_nuclear_transport,is_partner_category_other,is_partner_category_protein_folding,is_partner_category_protein_traffiking,is_partner_category_protein_turnover,is_partner_category_signaling,is_partner_category_translation,is_partner_category_vesicle_transport
9662,tf_isoform_ppis,101244,CREB5|2/3|08A12,CREB5,56717,KRTAP10-8,0,TFr07,102,B02,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# ETV2